In [ ]:
# %% Deep learning - Section 18.166
#    Code challenge 26: choose the parameters
#
#    1) Solve the problems listed below

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [2]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import seaborn             as sns
import copy
import torch.nn.functional as F
import pandas              as pd
import scipy.stats         as stats
import sklearn.metrics     as skm
import time
import sys
import imageio.v2          as imageio

from torch.utils.data                 import DataLoader,TensorDataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from scipy.stats                      import zscore
from sklearn.decomposition            import PCA
from scipy.signal                     import convolve2d
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')
plt.style.use('default')


In [ ]:
# %% Problem 0
#    Convolve an image of size 1x256x256 to produce a 1x252x84 result

# Parameters
chans_in  = 3          # (RGB)
chans_out = 15         # (feature maps)
img_size  = [256,256]  # (input img size)
krn_size  = 5          # (odd; 15 kernels of size 5x5)
stride    = (1,3)      # (no stride if =1)
padding   = 1          # (no padding if =0)

# Class instance
c = nn.Conv2d(chans_in,chans_out,krn_size,stride,padding)

# Generate image
img = torch.rand(1,chans_in,img_size[0],img_size[1])

# Run convolution and compute its actual shape
conv_img       = c(img)
size_empirical = torch.squeeze(conv_img).shape

# Compute the theoretical size of the result according to the formula
size_expected    = np.array([chans_out,0,0],dtype=int)
size_expected[1] = np.floor( (img_size[0]+2*padding-krn_size)/stride[0] ) + 1
size_expected[2] = np.floor( (img_size[1]+2*padding-krn_size)/stride[1] ) + 1

# Check
print(f'Expected size: {size_expected}')
print(f'Empirical size: {list(size_empirical)}')


In [ ]:
# %% Problem 1
#    Convolve an image of size 3x64x64 to produce a 10x28x28 result

# Parameters
chans_in  = 3          # (RGB)
chans_out = 10         # (feature maps)
img_size  = [64,64]    # (input img size)
krn_size  = 13         # (odd; 15 kernels of size 5x5)
stride    = (2,2)      # (no stride if =1)
padding   = 2          # (no padding if =0)

# Class instance
c = nn.Conv2d(chans_in,chans_out,krn_size,stride,padding)

# Generate image
img = torch.rand(1,chans_in,img_size[0],img_size[1])

# Run convolution and compute its actual shape
conv_img       = c(img)
size_empirical = torch.squeeze(conv_img).shape

# Compute the theoretical size of the result according to the formula
size_expected    = np.array([chans_out,0,0],dtype=int)
size_expected[1] = np.floor( (img_size[0]+2*padding-krn_size)/stride[0] ) + 1
size_expected[2] = np.floor( (img_size[1]+2*padding-krn_size)/stride[1] ) + 1

# Check
print(f'Expected size: {size_expected}')
print(f'Empirical size: {list(size_empirical)}')


In [ ]:
# %% Problem 2
#    Convolve an image of size 3x196x96 to produce a 5x66x49 result

# Parameters
chans_in  = 3         # (RGB)
chans_out = 5         # (feature maps)
img_size  = [196,96]  # (input img size)
krn_size  = 5         # (odd; 15 kernels of size 5x5)
stride    = (3,2)     # (no stride if =1)
padding   = 3         # (no padding if =0)

# Class instance
c = nn.Conv2d(chans_in,chans_out,krn_size,stride,padding)

# Generate image
img = torch.rand(1,chans_in,img_size[0],img_size[1])

# Run convolution and compute its actual shape
conv_img       = c(img)
size_empirical = torch.squeeze(conv_img).shape

# Compute the theoretical size of the result according to the formula
size_expected    = np.array([chans_out,0,0],dtype=int)
size_expected[1] = np.floor( (img_size[0]+2*padding-krn_size)/stride[0] ) + 1
size_expected[2] = np.floor( (img_size[1]+2*padding-krn_size)/stride[1] ) + 1

# Check
print(f'Expected size: {size_expected}')
print(f'Empirical size: {list(size_empirical)}')


In [ ]:
# %% Problem 3
#    Convolve an image of size 1x32x32 to produce a 6x28x28 result

# Parameters
chans_in  = 1         # (RGB)
chans_out = 6         # (feature maps)
img_size  = [32,32]   # (input img size)
krn_size  = 9         # (odd; 15 kernels of size 5x5)
stride    = (1,1)      # (no stride if =1)
padding   = 2         # (no padding if =0)

# Class instance
c = nn.Conv2d(chans_in,chans_out,krn_size,stride,padding)

# Generate image
img = torch.rand(1,chans_in,img_size[0],img_size[1])

# Run convolution and compute its actual shape
conv_img       = c(img)
size_empirical = torch.squeeze(conv_img).shape

# Compute the theoretical size of the result according to the formula
size_expected    = np.array([chans_out,0,0],dtype=int)
size_expected[1] = np.floor( (img_size[0]+2*padding-krn_size)/stride[0] ) + 1
size_expected[2] = np.floor( (img_size[1]+2*padding-krn_size)/stride[1] ) + 1

# Check
print(f'Expected size: {size_expected}')
print(f'Empirical size: {list(size_empirical)}')


In [ ]:
# %% Problem 4
#    Convolve an image of size 3x227x227 to produce a 96x55x55 result

# Parameters
chans_in  = 3          # (RGB)
chans_out = 96         # (feature maps)
img_size  = [227,227]  # (input img size)
krn_size  = 11          # (odd; 15 kernels of size 5x5)
stride    = (4,4)      # (no stride if =1)
padding   = 1          # (no padding if =0)

# Class instance
c = nn.Conv2d(chans_in,chans_out,krn_size,stride,padding)

# Generate image
img = torch.rand(1,chans_in,img_size[0],img_size[1])

# Run convolution and compute its actual shape
conv_img       = c(img)
size_empirical = torch.squeeze(conv_img).shape

# Compute the theoretical size of the result according to the formula
size_expected    = np.array([chans_out,0,0],dtype=int)
size_expected[1] = np.floor( (img_size[0]+2*padding-krn_size)/stride[0] ) + 1
size_expected[2] = np.floor( (img_size[1]+2*padding-krn_size)/stride[1] ) + 1

# Check
print(f'Expected size: {size_expected}')
print(f'Empirical size: {list(size_empirical)}')


In [ ]:
# %% Problem 5
#    Convolve an image of size 3x224x224 to produce a 64x224x224 result

# Parameters
chans_in  = 3          # (RGB)
chans_out = 64         # (feature maps)
img_size  = [224,224]  # (input img size)
krn_size  = 5          # (odd; 15 kernels of size 5x5)
stride    = (1,1)      # (no stride if =1)
padding   = 2          # (no padding if =0)

# Class instance
c = nn.Conv2d(chans_in,chans_out,krn_size,stride,padding)

# Generate image
img = torch.rand(1,chans_in,img_size[0],img_size[1])

# Run convolution and compute its actual shape
conv_img       = c(img)
size_empirical = torch.squeeze(conv_img).shape

# Compute the theoretical size of the result according to the formula
size_expected    = np.array([chans_out,0,0],dtype=int)
size_expected[1] = np.floor( (img_size[0]+2*padding-krn_size)/stride[0] ) + 1
size_expected[2] = np.floor( (img_size[1]+2*padding-krn_size)/stride[1] ) + 1

# Check
print(f'Expected size: {size_expected}')
print(f'Empirical size: {list(size_empirical)}')
